The goal of this section is loading the dataset, observing it and identifying possible following data cleaning steps

In [1]:
import pandas as pd

file_path = "../DATA/Online Retail Data Set.csv"
data = pd.read_csv(file_path, encoding='ISO-8859-1')

data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


### Step 1: Splitting InvoiceDate into Separate Columns
To facilitate seasonal and hourly analysis, the `InvoiceDate` column was split into four new columns:
- `Day`: The day of the month.
- `Month`: The month of the year.
- `Year`: The year of the transaction.
- `Hour`: The hour of the transaction.

This step ensures flexibility in analyzing patterns over time and during different parts of the day.

In [2]:
# Ensure InvoiceDate is in datetime format
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], errors='coerce')

# Create new columns for Day, Month, Year, and Hour
data['Day'] = data['InvoiceDate'].dt.day
data['Month'] = data['InvoiceDate'].dt.month
data['Year'] = data['InvoiceDate'].dt.year
data['Hour'] = data['InvoiceDate'].dt.hour

# Confirm the changes
print(data[['InvoiceDate', 'Day', 'Month', 'Year', 'Hour']].head())

          InvoiceDate   Day  Month    Year  Hour
0 2010-01-12 08:26:00  12.0    1.0  2010.0   8.0
1 2010-01-12 08:26:00  12.0    1.0  2010.0   8.0
2 2010-01-12 08:26:00  12.0    1.0  2010.0   8.0
3 2010-01-12 08:26:00  12.0    1.0  2010.0   8.0
4 2010-01-12 08:26:00  12.0    1.0  2010.0   8.0


### Step 2: Missing Values Analysis
To understand the quality of the data, a statistical analysis of missing values was performed:
- Counted the total missing values per column.
- Calculated the percentage of missing values relative to the total dataset size.

This helped prioritize which columns required attention.

In [3]:
# Analyze missing values
missing_values_stats = data.isnull().sum().reset_index()
missing_values_stats.columns = ['Column', 'MissingCount']
missing_values_stats['Percentage'] = (missing_values_stats['MissingCount'] / len(data)) * 100

# Display missing value statistics
print(missing_values_stats)

         Column  MissingCount  Percentage
0     InvoiceNo             0    0.000000
1     StockCode             0    0.000000
2   Description          1454    0.268311
3      Quantity             0    0.000000
4   InvoiceDate        308950   57.011417
5     UnitPrice             0    0.000000
6    CustomerID        135080   24.926694
7       Country             0    0.000000
8           Day        308950   57.011417
9         Month        308950   57.011417
10         Year        308950   57.011417
11         Hour        308950   57.011417


### Step 3: Handling Discounts
Rows where `UnitPrice` is zero were identified as discounts. There are approximately 100 discount lines in the dataset. These rows are negligible compared to the dataset size (541,910 rows) and can be removed for simplicity unless further analysis of discounts is required.


In [4]:
# Identify discount lines (UnitPrice = 0)
discount_lines = data[data['UnitPrice'] == 0]

# Count and examine sample discount lines
print(f"Number of discount rows: {len(discount_lines)}")
print(discount_lines.head())

Number of discount rows: 2515
     InvoiceNo StockCode Description  Quantity         InvoiceDate  UnitPrice  \
622     536414     22139         NaN        56 2010-01-12 11:52:00        0.0   
1970    536545     21134         NaN         1 2010-01-12 14:32:00        0.0   
1971    536546     22145         NaN         1 2010-01-12 14:33:00        0.0   
1972    536547     37509         NaN         1 2010-01-12 14:33:00        0.0   
1987    536549    85226A         NaN         1 2010-01-12 14:34:00        0.0   

      CustomerID         Country   Day  Month    Year  Hour  
622          NaN  United Kingdom  12.0    1.0  2010.0  11.0  
1970         NaN  United Kingdom  12.0    1.0  2010.0  14.0  
1971         NaN  United Kingdom  12.0    1.0  2010.0  14.0  
1972         NaN  United Kingdom  12.0    1.0  2010.0  14.0  
1987         NaN  United Kingdom  12.0    1.0  2010.0  14.0  


### Step 4: Handling Negative Quantities
Rows with negative values in the `Quantity` column were identified as potential refunds or errors. These rows were:
- Counted to determine the extent of the issue.
- Further analyzed to check patterns (e.g., refund-related invoices).

These rows can either be excluded or categorized depending on the analysis objectives.

In [5]:
# Identify rows with negative quantities
negative_quantity_lines = data[data['Quantity'] < 0]

# Count and display sample negative quantity rows
print(f"Number of rows with negative quantities: {len(negative_quantity_lines)}")
print(negative_quantity_lines.head())


Number of rows with negative quantities: 10624
    InvoiceNo StockCode                       Description  Quantity  \
141   C536379         D                          Discount        -1   
154   C536383    35004C   SET OF 3 COLOURED  FLYING DUCKS        -1   
235   C536391     22556    PLASTERS IN TIN CIRCUS PARADE        -12   
236   C536391     21984  PACK OF 12 PINK PAISLEY TISSUES        -24   
237   C536391     21983  PACK OF 12 BLUE PAISLEY TISSUES        -24   

            InvoiceDate  UnitPrice  CustomerID         Country   Day  Month  \
141 2010-01-12 09:41:00      27.50     14527.0  United Kingdom  12.0    1.0   
154 2010-01-12 09:49:00       4.65     15311.0  United Kingdom  12.0    1.0   
235 2010-01-12 10:24:00       1.65     17548.0  United Kingdom  12.0    1.0   
236 2010-01-12 10:24:00       0.29     17548.0  United Kingdom  12.0    1.0   
237 2010-01-12 10:24:00       0.29     17548.0  United Kingdom  12.0    1.0   

       Year  Hour  
141  2010.0   9.0  
154  2010.0

### Step 5: Exploring Product Purchase Patterns
To understand customer behavior:
- Grouped purchases by `CustomerID` and `InvoiceNo` to analyze products purchased together.
- This analysis helps in identifying commonly bought product combinations, which can guide marketing strategies such as personalized discounts or bundling recommendations.

In [6]:
# Group by CustomerID and InvoiceNo to find product combinations
grouped_data = data.groupby(['CustomerID', 'InvoiceNo'])['Description'].apply(list)

# Display a sample of product combinations
print(grouped_data.head())

CustomerID  InvoiceNo
12346.0     541431                        [MEDIUM CERAMIC TOP STORAGE JAR]
            C541433                       [MEDIUM CERAMIC TOP STORAGE JAR]
12347.0     537626       [BLACK CANDELABRA T-LIGHT HOLDER, AIRLINE BAG ...
            542237       [PINK NEW BAROQUECANDLESTICK CANDLE, BLUE NEW ...
            549222       [AIRLINE BAG VINTAGE JET SET WHITE, AIRLINE BA...
Name: Description, dtype: object
